In [1]:
import os
import datetime

from time import perf_counter
from tqdm import tqdm

# from fetchbim import Client, Family
# from fetchbim.utils import truncate
# from fetchbim.teams import sync_complete_msg
from notion_client import Client as NotionClient

In [2]:
def get_all_pages(client, database_id: str, **kwargs) -> list:
    results = []
    cursor = None
    while True:
        response = client.databases.query(
            database_id=database_id, start_cursor=cursor
        )
        results.extend(response["results"])
        if response["has_more"] is False:
            break
        else:
            cursor = response["next_cursor"]
            
    return results

In [3]:
notion = NotionClient(auth=os.environ["NOTION_KEY"])

In [4]:
notion.options.timeout_ms = 100000

In [5]:
mfg = get_all_pages(notion, database_id="8ef28b92d8104e7faf2560198854d741")

In [20]:
properties = dict()
for page in tqdm(mfg):
    page_id = page['id']
    prop = page['properties']['Potential Product Lines']['rich_text']
    if prop:
        value = prop[0]['text']['content']
        if ',' in value:
            new_value = '\n'.join([x.strip() for x in value.split(',')])
            properties['Potential Product Lines'] = {'rich_text': [{'text': {'content': new_value}}]}
            notion.pages.update(page_id=page_id, properties=properties)

100%|██████████| 791/791 [00:42<00:00, 18.62it/s] 
